Optional Usage of venv to download and install the required packages

```bash
python3 -m venv venv
source venv/bin/activate
pip install -r requirements.txt
```

In [1]:
# Exports combined data to data/ folder by default. Change to False otherwise
EXPORT_DATA = True
# Exports model to model/ folder by default. Change to False otherwise
EXPORT_MODELS = True

# Import packages and functions

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import pyarrow.parquet as pq

np.random.seed(59)

# EDA and Data-Preprocessing

In [6]:
agent_df = pd.read_parquet('data/nus_agent_info_df.parquet', engine='pyarrow')
client_df = pd.read_parquet('data/nus_client_info_df.parquet', engine='pyarrow')
policy_df = pd.read_parquet('data/nus_policy_info_df.parquet', engine='pyarrow')

## Agent Dataset

We drop the following columns:
1. cluster: Criteria for old segment is unknown

In [7]:
agent_df = agent_df.drop(columns=["cluster"])

We expand `agent_product_expertise`. In other words, we create new variables, with each column indicating if the agent is comfortable selling the product

In [8]:
agent_df_exploded = agent_df.explode("agent_product_expertise")
agent_df_encoded = pd.get_dummies(agent_df_exploded, columns=["agent_product_expertise"])
agent_df = agent_df_encoded.groupby("agntnum").max().reset_index()

After some inspection, we realized there are some values that is labelled `U`, which means it is unknown/undisclosed. Hence we decided to remove those data points

In [9]:
agent_df = agent_df.dropna()
agent_df = agent_df[agent_df["agent_age"] != "U"]
agent_df = agent_df[agent_df["agent_gender"] != "U"]
agent_df = agent_df[agent_df["agent_marital"] != "U"]

Finally, we one-hot-encode the categorical variables

In [10]:
def encode_categorical_variables(df, exclude_columns=None):
    if exclude_columns is None:
        exclude_columns = []
    
    # Select all categorical columns (exclude specified columns)
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns
    categorical_columns = [col for col in categorical_columns if col not in exclude_columns]
    
    # Apply one-hot encoding using get_dummies for all categorical columns
    df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)  # drop_first to avoid multicollinearity
    
    return df_encoded

agent_encoded_df = encode_categorical_variables(agent_df, exclude_columns=["agntnum"])
final_agent_df = agent_encoded_df.apply(lambda col: col.astype(int) if col.dtype == 'bool' else col)

## Client Dataset

Following the same analysis, we drop the data points where there is unknown values. We also converted the dob to the person's age and decided not to use the postal code due to privacy concerns.

In [11]:
client_df = pd.read_parquet('data/nus_client_info_df.parquet', engine='pyarrow')
client_df = client_df.dropna()
client_df['cltdob'] = pd.to_datetime(client_df['cltdob'])
today = pd.to_datetime('today')
client_df['age'] = ((today - client_df['cltdob']).dt.days / 365.25).round()
client_df = client_df[client_df["marryd"] != "U"]
client_df = client_df.drop(columns=["cltdob", "cltpcode", "household_size", "family_size"])
client_df = client_df[client_df["household_size_grp"] != "HH0_missing"]
client_encoded_df = encode_categorical_variables(client_df, exclude_columns=["secuityno", "economic_status"])
final_client_df = client_encoded_df.apply(lambda col: col.astype(int) if col.dtype == 'bool' else col)

## Policy Dataset

Same as the previous 2 datasets, we remove the unknown values, and convert the `occdate` variable to `time_elapsed`. We also dropped a few columns due to either lack of information in the data dictionary, or because their values are all the same

In [12]:
policy_df = pd.read_parquet('data/nus_policy_info_df.parquet', engine='pyarrow')
policy_df = policy_df.dropna()
policy_df['occdate'] = pd.to_datetime(policy_df['occdate'])
today = pd.to_datetime('today')
policy_df['time_elapsed'] = ((today - policy_df['occdate']).dt.days / 365.25).round()
policy_df = policy_df.drop(columns=["occdate", "flg_lapsed", "flg_main", "flg_rider", "flg_cancel", "flg_converted", "product_grp"])
policy_encoded_df = encode_categorical_variables(policy_df, exclude_columns=["secuityno", "agntnum", "chdrnum"])
final_policy_df = policy_encoded_df.apply(lambda col: col.astype(int) if col.dtype == 'bool' else col)
final_policy_df ## Output is removed for data privacy reasons

,chdrnum,agntnum,secuityno,annual_premium,flg_inforce,flg_expire,time_elapsed,product_prod_2,product_prod_4,product_prod_6,...,cust_age_at_purchase_grp_AG04_30to34,cust_age_at_purchase_grp_AG05_35to39,cust_age_at_purchase_grp_AG06_40to44,cust_age_at_purchase_grp_AG07_45to49,cust_age_at_purchase_grp_AG08_50to54,cust_age_at_purchase_grp_AG09_55to59,cust_age_at_purchase_grp_AG10_60up,cust_tenure_at_purchase_grp_TNR3_1to4yr,cust_tenure_at_purchase_grp_TNR4_4to8yr,cust_tenure_at_purchase_grp_TNR5_8yrup
0,PID:281,AIN:62,CIN:6957,0.0,1,0,6.0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,PID:280,AIN:63,CIN:2161,7.0,1,0,1.0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,PID:2577,AIN:63,CIN:16605,423.0,1,0,0.0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
3,PID:2578,AIN:63,CIN:16605,217.0,1,0,0.0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
4,PID:305,AIN:63,CIN:7917,432.0,1,0,0.0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29498,PID:22084,AIN:6527,CIN:15419,4574.4,1,0,12.0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
29499,PID:13706,AIN:3472,CIN:17337,267.0,1,0,16.0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
29500,PID:25287,AIN:7852,CIN:13015,0.0,0,0,25.0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
29501,PID:26334,AIN:8316,CIN:2239,1152.0,1,0,15.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


Finally, we merge all the datasets together

In [13]:
client_policy_df = final_policy_df.merge(final_client_df, how="inner", on="secuityno")
combined_df = client_policy_df.merge(final_agent_df, how="inner", on="agntnum")


In [14]:
# Saving of files
if EXPORT_DATA == True:
    combined_df.to_csv("data/combined_info.csv", index=False)

# ML Modelling

## Feature Engineering

In [15]:
# Read combined df
if EXPORT_DATA == True:
    df = pd.read_csv('data/combined_info.csv')
else:
    df = combined_df
agent_df = pd.read_parquet('data/nus_agent_info_df.parquet', engine="pyarrow")

# set variables to be used in the model
client_vars = ['cust_age_at_purchase_grp_AG01_lt20',
        'cust_age_at_purchase_grp_AG07_45to49',
       'cust_age_at_purchase_grp_AG08_50to54',
       'cust_age_at_purchase_grp_AG09_55to59',
       'cust_age_at_purchase_grp_AG10_60up','economic_status', 'age',
       'cltsex_M', 'marryd_M', 'marryd_P', 'marryd_S', 'marryd_W',
       'race_desc_map_Indian', 'race_desc_map_Malay', 'race_desc_map_Others',
       'household_size_grp_HH2_40to80', 'household_size_grp_HH3_80to100',
       'household_size_grp_HH4_100to120', 'household_size_grp_HH5_120up',
       'family_size_grp_FS2_20to40', 'family_size_grp_FS3_40to60',
       'family_size_grp_FS4_60to80', 'family_size_grp_FS5_80up']

labels_to_predict = ['agent_age', 'agent_gender_M', 'agent_marital', 'agent_tenure', 'cnt_converted_cat', 'annual_premium_cnvrt_cat', 'agntnum'] # 'cnt_converted', 'annual_premium_cnvrt',

## Functions to convert numerical to ordinal categories (in numbers) 
def cnt_converted_to_cat(num):
    """Convert cnt_converted into category bins of size 500."""
    return int(num // 500)

def annual_premium_cnvrt_to_cat(num):
    """Convert annual_premium_cnvrt into category bins of size 4000000."""
    return int(num // 4000000) 

## Data Preprocessing 
## Preprocess client columns
columns_to_scale = ['economic_status', 'age']
scaler = StandardScaler()
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

## Preprocess label columns
df = df.merge(agent_df[['agntnum','agent_marital']], on='agntnum', how='left')
df['cnt_converted_cat'] = df['cnt_converted'].apply(cnt_converted_to_cat).astype(int)
df['annual_premium_cnvrt_cat'] = df['annual_premium_cnvrt'].apply(annual_premium_cnvrt_to_cat).astype(int)
## Split data into X and y_multiple
X = df[client_vars]
y_multiple = df[labels_to_predict]

# Split dataset into testing and training (0.2 test data)
X_train, X_test, y_train_multiple, y_test_multiple = train_test_split(X, y_multiple, test_size=0.2, random_state=59)


## Train and Evaluate Individual Models

In [16]:
models = {
        'agent_age': {
                'model': RandomForestClassifier(max_features=None, n_jobs=-1) 
                },
        'agent_gender_M' : {
                'model': RandomForestClassifier(max_features=None, n_jobs=-1)
                },
        'agent_tenure' : {
                'model': RandomForestClassifier(max_features=None, n_jobs=-1) 
                },
        'agent_marital' : {
                'model': RandomForestClassifier(max_features=None, n_jobs=-1)
                },
        'cnt_converted_cat' : {
                'model': RandomForestClassifier(max_features=None, n_jobs=-1)
                },
        'annual_premium_cnvrt_cat' : {
                'model': RandomForestClassifier(max_features=None, n_jobs=-1)
                },
        }

# Initialise dataframe for concatenating y_pred for each label
y_pred_multiple = pd.DataFrame()

# Intialise fitted_model dictionary 
fitted_models = {}

# Train, fit and predict for each label and corresponding model
for label_name, dct in models.items():
        print(f"Training model for {label_name}")

        # Define y_train
        y_train = y_train_multiple[label_name]
        
        # Cross validation score of model
        model = dct['model']
        scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
        model.fit(X_train, y_train)

        # Compute mean and standard deviation
        mean_score = np.mean(scores)
        std_score = np.std(scores)
        print(f"Label: {label_name}")
        print(f"Cross-validation accuracy scores: {scores}")
        print(f"Mean accuracy: {mean_score:.4f}")
        print(f"Standard deviation: {std_score:.4f}")
        print()

        # Save model 
        fitted_models[label_name] = model        

Training model for agent_age


/Users/emilyng/Documents/Projects/singlife_fa_recommendation/myvenv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Label: agent_age
Cross-validation accuracy scores: [0.36171797 0.36271102 0.3503849  0.34765334 0.35013658]
Mean accuracy: 0.3545
Standard deviation: 0.0064

Training model for agent_gender_M
Label: agent_gender_M
Cross-validation accuracy scores: [0.66683217 0.6633565  0.67221257 0.66898436 0.6642662 ]
Mean accuracy: 0.6671
Standard deviation: 0.0032

Training model for agent_tenure


/Users/emilyng/Documents/Projects/singlife_fa_recommendation/myvenv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Label: agent_tenure
Cross-validation accuracy scores: [0.32373386 0.3182721  0.31686119 0.30370002 0.32331761]
Mean accuracy: 0.3172
Standard deviation: 0.0073

Training model for agent_marital
Label: agent_marital
Cross-validation accuracy scores: [0.67428004 0.65566038 0.67072262 0.67022597 0.67146759]
Mean accuracy: 0.6685
Standard deviation: 0.0066

Training model for cnt_converted_cat


/Users/emilyng/Documents/Projects/singlife_fa_recommendation/myvenv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Label: cnt_converted_cat
Cross-validation accuracy scores: [0.64920556 0.64995035 0.64117209 0.63695058 0.6548299 ]
Mean accuracy: 0.6464
Standard deviation: 0.0065

Training model for annual_premium_cnvrt_cat
Label: annual_premium_cnvrt_cat
Cross-validation accuracy scores: [0.97467726 0.97269116 0.97566427 0.97491929 0.97616091]
Mean accuracy: 0.9748
Standard deviation: 0.0012



In [ ]:
# Testing the model
# Predict
for label_name in fitted_models:
    model = fitted_models[label_name]
    y_test = y_test_multiple[label_name]
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{label_name} accuracy score: {acc}")

    # append column to y_pred_multiple
    y_pred_multiple[label_name] = y_pred

agent_age accuracy score: 0.3926514399205561
agent_gender_M accuracy score: 0.6796425024826217


## Combine Model Results 

### One hot encode label `agent_marital`
- `agent_marital` is currently a categorical datapoint. To calcualte the cosine similarity score, it needs to be one hot encoded for an accurate representation of the data

In [16]:
def encode_categorical_variables(df, exclude_columns=None):
    if exclude_columns is None:
        exclude_columns = []
    
    # Select all categorical columns (exclude specified columns)
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns
    categorical_columns = [col for col in categorical_columns if col not in exclude_columns]
    
    # Apply one-hot encoding using get_dummies for all categorical columns
    df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)  # drop_first to avoid multicollinearity
    
    return df_encoded

# This is All the agents data
y_multiple = encode_categorical_variables(y_multiple, exclude_columns=["agntnum"])
y_multiple = y_multiple.apply(lambda col: col.astype(int) if col.dtype == 'bool' else col)
y_multiple = y_multiple[[col for col in y_multiple.columns if col != 'agntnum'] + ['agntnum']]

# This is the agent data test set
y_test_multiple = encode_categorical_variables(y_test_multiple, exclude_columns=["agntnum"])
y_test_multiple = y_test_multiple.apply(lambda col: col.astype(int) if col.dtype == 'bool' else col)
y_test_multiple = y_test_multiple[[col for col in y_test_multiple.columns if col != 'agntnum'] + ['agntnum']]

# This is the predicted agent data, to be compared with the test set
y_pred_multiple = encode_categorical_variables(y_pred_multiple, exclude_columns=["agntnum"])
y_pred_multiple = y_pred_multiple.apply(lambda col: col.astype(int) if col.dtype == 'bool' else col)

In [17]:
# Check if all columns in y_test_multiple (except last column 'agntnum') is the same as y_pred_multiple.
# If it is not the same, it may be due to one hot encoding of less categorical variables than expected due to the prediction model
if 'agent_marital_W' not in y_pred_multiple.columns:
    y_pred_multiple['agent_marital_W'] = 0
    y_pred_multiple = y_pred_multiple[y_test_multiple.columns[:-1]]

# This ensures all columns in y_pred_multiple is in the same order as y_test_multiple (except last column 'agntnum') 
y_pred_multiple = y_pred_multiple[y_test_multiple.columns[:-1]]

### Predict Top 10 Agents
Similarity scores are calculated using the cosine similarity score. The top 10 agents are then predicted based on the similarity scores

In [18]:
top_k = 10
num_columns = len(y_pred_multiple.columns)  # This number is not equals to labels_to_predict agent_marital has been one hot encoded

print(f"Predicting top {top_k} agents based on {num_columns} columns")
# Calculate Cosine Similarity Matrix with the first 
similarity_matrix = cosine_similarity(y_pred_multiple.iloc[:, :num_columns], y_multiple.iloc[:, :num_columns]) 

# Get Top 10 Most Similar Rows for Each Row in y_pred_multiple
top_10_matches = [y_multiple.iloc[np.argsort(-row)[:top_k], num_columns].tolist() for row in similarity_matrix]

# Add Results to y_pred_multiple as a new column
y_pred_multiple["top10_agntnum"] = top_10_matches

# Display Output ## Output is removed for data privacy reasons
y_pred_multiple.head()

Predicting top 10 agents based on 8 columns


,agent_age,agent_gender_M,agent_tenure,cnt_converted_cat,annual_premium_cnvrt_cat,agent_marital_M,agent_marital_S,agent_marital_W,top10_agntnum
0,39.0,0,143.0,0,0,1,0,0,"[AIN:6838, AIN:3391, AIN:3391, AIN:7325, AIN:7..."
1,31.0,0,39.0,0,0,0,1,0,"[AIN:506, AIN:9110, AIN:3570, AIN:3570, AIN:35..."
2,53.0,0,9.0,0,0,0,0,0,"[AIN:2649, AIN:2649, AIN:9272, AIN:466, AIN:46..."
3,40.0,1,36.0,7,0,1,0,0,"[AIN:1291, AIN:1828, AIN:1828, AIN:1828, AIN:1..."
4,27.0,0,13.0,0,0,0,1,0,"[AIN:1270, AIN:4903, AIN:10, AIN:10, AIN:4381,..."


### Evaluate Combined Model

In [19]:
y_true = np.ones(len(y_test_multiple["agntnum"]))  # Since we are checking if "agntnum" exists, all should be 1
y_compared = [1 if agnt in top10 else 0 for agnt, top10 in zip(y_test_multiple["agntnum"], y_pred_multiple["top10_agntnum"])]

# Compute Confusion Matrix
cm = confusion_matrix(y_true, y_compared)

# Compute Accuracy
accuracy = accuracy_score(y_true, y_compared)

# 🚀 Print Results
print("Confusion Matrix:\n", cm, "\n")
print(f"Accuracy:\n{accuracy:.4f}")

Confusion Matrix:
 [[   0    0]
 [3375 1660]] 

Accuracy:
0.3297


### Further Interpretations of Results

In [20]:
## To observe how the model performs on each row:

# Assign true false column to prediction df
y_pred_multiple['agnt_predicted'] = y_compared

# concat true values df (test set) with prediction df
y_compare_pred_test = pd.concat([y_test_multiple.reset_index(drop=True),y_pred_multiple.reset_index(drop=True)],axis=1)

# look at rows where prediction is wrong ## Output is removed for data privacy reasons
y_compare_pred_test[y_compare_pred_test['agnt_predicted']==0]

,agent_age,agent_gender_M,agent_tenure,cnt_converted_cat,annual_premium_cnvrt_cat,agent_marital_M,agent_marital_S,agent_marital_W,agntnum,agent_age,agent_gender_M,agent_tenure,cnt_converted_cat,annual_premium_cnvrt_cat,agent_marital_M,agent_marital_S,agent_marital_W,top10_agntnum,agnt_predicted
0,39.0,1,92.0,0,0,1,0,0,AIN:5552,39.0,0,143.0,0,0,1,0,0,"[AIN:6838, AIN:3391, AIN:3391, AIN:7325, AIN:7...",0
1,30.0,0,9.0,0,0,0,1,0,AIN:505,31.0,0,39.0,0,0,0,1,0,"[AIN:506, AIN:9110, AIN:3570, AIN:3570, AIN:35...",0
3,46.0,0,106.0,3,0,0,1,0,AIN:2432,40.0,1,36.0,7,0,1,0,0,"[AIN:1291, AIN:1828, AIN:1828, AIN:1828, AIN:1...",0
4,26.0,0,39.0,0,0,1,0,0,AIN:8477,27.0,0,13.0,0,0,0,1,0,"[AIN:1270, AIN:4903, AIN:10, AIN:10, AIN:4381,...",0
5,57.0,0,31.0,0,0,1,0,0,AIN:4453,25.0,1,33.0,7,0,0,1,0,"[AIN:2974, AIN:2974, AIN:2974, AIN:2974, AIN:2...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5029,36.0,0,20.0,0,0,1,0,0,AIN:1649,36.0,0,20.0,0,0,1,0,0,"[AIN:7601, AIN:3357, AIN:3357, AIN:3357, AIN:3...",0
5030,41.0,1,15.0,1,0,0,1,0,AIN:251,32.0,1,54.0,8,0,0,1,0,"[AIN:417, AIN:417, AIN:417, AIN:417, AIN:417, ...",0
5032,35.0,0,39.0,0,0,0,1,0,AIN:979,35.0,0,16.0,0,0,0,1,0,"[AIN:542, AIN:978, AIN:978, AIN:9244, AIN:542,...",0
5033,46.0,1,40.0,0,0,0,1,0,AIN:5491,40.0,1,41.0,0,0,0,1,0,"[AIN:7700, AIN:9792, AIN:871, AIN:871, AIN:242...",0


# Exporting ML Model

In [21]:
if EXPORT_MODELS == True:
    for label_name, model in fitted_models.items():
        print(f"Saving model for {label_name}")
        with open(f"model/{label_name}_model.pkl", 'wb') as file:
            pickle.dump(model, file)

Saving model for agent_age
Saving model for agent_gender_M
Saving model for agent_tenure
Saving model for agent_marital
Saving model for cnt_converted_cat
Saving model for annual_premium_cnvrt_cat
